## Still Drafting

### Working copy of a script to check if a path for storing data exists, if the desired time range is already downloaded, and if not, gets and saves data from IRIS, then plots RSAM, Spectrogram, PPSDs, and Temporal Plots for each day within the specified time range. Ideally allows for inputting larger time ranges and getting 1 plot per day. 
### Currently works as normal for already saved data, but still some issues with clean/full outputs of the data it downloads via the script. Polished version TBD

In [ ]:
# fixing reading and displaying plots as each day's data is processed but error with downloading section

import obspy
from obspy import UTCDateTime, read
from obspy.signal import PPSD
import numpy as np
import matplotlib.pyplot as plt
from obspy.imaging.cm import pqlx
import os
from obspy.clients.fdsn import Client
from datetime import timedelta
from obspy.clients import fdsn
import warnings

# define start and end time for the data range
start_date = UTCDateTime(2023, 8, 25)  # Start time
end_date = UTCDateTime(2023, 8, 27)  # End time
delta = 86400  # one day
download_delta = 1800  # 30 minutes per data chunk
stas = [2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 2312, 2313, 2314, 2315, 2316, 
        'G2301', 'G2302', 'G2303', 'G2304', 'G2305', 'G2306', 'G2307', 'G2308', 'G2309', 'G2310', 'G2311',
        'G2312', 'G2313', 'G2314', 'G2315', 'G2316'] 

stas = ['ZE.2304..GPZ']

# define path where .mseed files are stored
path = 'C:/Users/zzawol/Documents/seismic-data-iris/seismic_data/NO2304/GPZ'

# IRIS credentials
username = 'zoe_zawol@partner.nps.gov'
password = 'rJXKed4LZUHUE05g'

# set station info
nowsta = stas[0]
network, station, location, channel = nowsta.split('.')
print(f"Using station: {nowsta}")

# define the file pattern for reading mseed files
file_pattern = f'{path}/{network}.{station}.*.{channel}*.mseed'

# function to check if files for a specific date are present in the directory
def check_files_for_day(path, day_start_time):
    files = os.listdir(path)
    required_files = set()

    # check if any files for the given day exist
    day_str = day_start_time.strftime('%Y%m%d')
    for file in files:
        if day_str in file:  # check if the file is for this day
            return True  # files exist for this day

    return False  # files are missing for this day

# IRIS client setup
DATASELECT = 'http://service.iris.edu/ph5ws/dataselect/1'
c = fdsn.client.Client(service_mappings={'dataselect': DATASELECT})
c.set_credentials(username, password)

# check if path exists and if files are within the given time range -- add create path in else
if os.path.isdir(path):
    print("Path exists. Checking for files within the time range...")

    # loop through each day in the date range
    current_date = start_date
    while current_date < end_date:
        print(f"\nProcessing data for {current_date} to {current_date + delta}")

        # store traces for the full day
        day_tr = []

        # check if data for the current day exists
        if check_files_for_day(path, current_date):
            print(f"Files for {current_date.strftime('%Y%m%d')} already exist. Reading data from disk.")
            try:
                # read all data for the current day in one go
                S = obspy.read(file_pattern, starttime=current_date, endtime=current_date + delta)
                S.merge(method=1)
                print("Resampling...")
                S.resample(100) # resample for PPSD
                day_tr.extend(S) # add to list
                
            except Exception as e:
                print(f"Error reading data for {current_date}: {e}")
        else:
            # if data for current day is missing, download it
            print(f"Downloading missing data for {current_date.strftime('%Y%m%d')}")
            tNow = current_date
            req = [] #UNSURE??
            while tNow < current_date + delta:
                try:
                    print(f"  Downloading data for {tNow} to {tNow + download_delta}")
                    S = c.get_waveforms(network, station, location, channel, tNow, tNow + download_delta)
                    if S:
                        # set up metadata
                        client = Client("IRISPH5")
                        inv = client.get_stations(network=network, station=station, location=location, channel=channel, level='response')
                        print("Resampling for download - 350 Hz")
                        S.resample(350)  # resampled rate for final analysis. See Nyquist rules.
                        S.merge(method=1) # merge traces to avoid duplicates

                        # plotting RSAM and spectrogram
                        f1 = waveformUtils.multiDaySpectrogram(S, averageLength=3600, fftLength=60, minFreq=0.5, maxFreq=150, cmap='turbo',
                                      dateLimits=None,
                                      vmin=0.4, vmax=0.9, plotAverage=True)
                        f1.show()

                        # # prefiltering for instrument's frequency bandwidth, do before preprocessing
                        # for tr in S: # adjust to not read below 0.1 Hz?***
                        #     pre_filt = [0.1, 1, 100, 150] # 4 corner frequencies # values TBD
                        #     tr.remove_response(inventory=inv, pre_filt=pre_filt, plot=True) # , output="DISP", water_level=60, plot=False)  
    
                        print("Resampling for PPSD analysis - 300 Hz")
                        S.resample(300)
                        
                        # save the data to files
                        for tr in S:
                            filename = f'{path}/{tr.id}_{tr.stats.starttime.strftime("%Y%m%d%H%M%S")}.mseed'
                            tr.write(filename)
                            #print(f"Data saved as {filename}")
                        day_tr.extend(S)  # store the traces for the full day
                    else:
                        print(f"No data returned for {tNow} to {tNow + download_delta}")
                except Exception as e:
                    print(f"Error downloading data for {tNow} to {tNow + download_delta}: {e}")
                tNow += download_delta  # increment by 30 minutes to next time chunk

        # now that the full day's data has been gathered, proceed with analysis and plotting
        if day_tr:
            # # merge traces to avoid duplicates  ### could leave these here to avoid repeating commands - but may not have worked for tr
            # day_tr.merge(method=1)

            # # resample the data
            # print('Resampling...')
            # day_tr.resample(100)  # Resampled rate for final analysis

            # create PPSD object and add traces
            ppsd = None
            for tr in day_tr:
                if ppsd is None:
                    ppsd = PPSD(tr.stats, metadata=inv, ppsd_length=25*60, period_limits=(0.005, 10.0)) #3 try this length, if not try 20, or min(len(tr))
                    print(f"PPSD length: {25*60}")
                    print(f"Min trace length: {np.min(len(tr))}")
                ppsd.add(tr)
                #print('Trace added')

            # plot the PPSD for the current day
            if ppsd is not None and len(ppsd.times_processed) > 0:
                print(f"Data accumulated for {(end_date-start_date)/(60*60*24)}-day periods starting on {current_date.strftime('%Y%m%d')}")
                print("Processed times:", ppsd.times_processed[:10])  # see first 10 times processed

                # plot the PPSD
                plt.figure(figsize=(10, 6))
                ppsd.plot(period_lim=(0.005,10),cmap=pqlx)
                psd_values = ppsd.psd_values
                print("Shape of psd values: ", np.shape(psd_values))

                # plotting time series of PSD values
                ppsd.plot_temporal([0.1, 1, 10])

                plt.show()  # show the plot for the current day
                plt.close()  # close plot to avoid displaying it in each iteration

        # move to the next day
        current_date += delta

else:
    print(f"Path does not exist. Downloading data from IRIS and creating path...")
    DATASELECT = 'http://service.iris.edu/ph5ws/dataselect/1'
    c = fdsn.client.Client(service_mappings={'dataselect': DATASELECT})
    c.set_credentials(username, password)

    # create directory if it doesn't exist
    os.makedirs(path, exist_ok=True)

    # download data for the entire time range
    tNow = start_date  # start time
    while tNow < end_date:
        print(f"Downloading data for {tNow} to {tNow + delta}")
        try:
            #warnings.filterwarnings("ignore", message=".*encoding.*") # suppress download process warnings - otherwise shows up a ton
            S = c.get_waveforms(network, station, location, channel, tNow, tNow + download_delta)
            if S:
                S.merge(method=1)
                S.resample(350)
                # save the data to files
                for tr in S:
                    filename = '{}/{}_{}.mseed'.format(path, tr.id, tr.stats.starttime.strftime('%Y%m%d%H%M%S'))
                    tr.write(filename)
                    #print(f"Data saved as {filename}")
            else:
                print(f"No data returned for {tNow} to {tNow + download_delta}")
        except Exception as e:
            print(f"Error downloading data for {tNow} to {tNow + download_delta}: {e}")
        tNow += download_delta
